Installing the packages and loading the data from Google Drive

In [1]:
! pip install -q lalsuite
! pip install -q gwpy
! pip install -q pycbc
# -- Click "restart runtime" in the runtime menu
! pip install matplotlib==3.1.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gwpy 2.1.3 requires matplotlib>=3.3.0, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
from google.colab import drive
from gwpy.timeseries import TimeSeries
import pycbc
from pycbc import frame

drive.mount('/content/drive')

pycbc_strain = pycbc.types.load_timeseries('/content/drive/MyDrive/GW_noisedata-2.npy')


Mounted at /content/drive


Loading the packages needed for Task 4


In [4]:
from pycbc.filter import matched_filter
import numpy
from pycbc.waveform import get_td_waveform
import pylab
from pycbc import frame
from pycbc.filter import matched_filter
import numpy
import pylab
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation

1. For this first part we can create an arange of the solar masses (from 5 to 10 increasing by 0.1 at each point) as well as calculating the sample rate and the duration.

In [3]:
import numpy as ar

m = ar.arange(5,10,0.1)
m1 = m2 = m
print(m)

samplerate = pycbc_strain.sample_rate
duration = pycbc_strain.duration
print('This is the sample rate:', samplerate, 'and the duration:', duration)

[5.  5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6.  6.1 6.2 6.3 6.4 6.5
 6.6 6.7 6.8 6.9 7.  7.1 7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9 8.  8.1
 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9 9.  9.1 9.2 9.3 9.4 9.5 9.6 9.7
 9.8 9.9]
This is the sample rate: 4096.0 and the duration: 256.0


Now we make iterations of thoses masses with the arange and a for loop. containing. We also create a template of the cross polarization using the same method as for Task 2/ Task 3.

In [5]:
listsnr = ar.zeros((len(m), 2), float)

for i in range(0, len(m)):
  hp = get_td_waveform(approximant="SEOBNRv4_opt",
                     mass1=m[i],
                     mass2=m[i],
                     delta_t=pycbc_strain.delta_t,
                     f_lower=15)

We can condition the data and make the PSD

In [6]:
pycbc_strain = highpass(pycbc_strain, 15.0)
pycbc_strain = resample_to_delta_t(pycbc_strain, 1.0/2048)
conditioned = pycbc_strain.crop(3,3)
# Resize the vector to match our data
hp.resize(len(conditioned))

template = hp.cyclic_time_shift(hp.start_time)

AttributeError: ignored

In [7]:
from pycbc.psd import interpolate, inverse_spectrum_truncation
psd = pycbc_strain.psd(4)
# Estimate the power spectral density

# We use 4 second samples of our time series since the data length is 128 which is divisible by 4
psd = conditioned.psd(4)

# Now that we have the psd we need to interpolate it to match our data
# and then limit the filter length of 1 / PSD. After this, we can
# directly use this PSD to filter the data in a controlled manner
psd = interpolate(psd, conditioned.delta_f)

# 1/PSD will now act as a filter with an effective length of 4 seconds
# Since the data has been highpassed above 15 Hz, and will have low values
# below this we need to inform the function to not include frequencies
# below this frequency. 
psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),
                                  low_frequency_cutoff=15)

Again we use a for loop because of the different masses to show the max SNR for each mass. 

In [8]:
snr = matched_filter(template, conditioned,
                     psd=psd, 
                     low_frequency_cutoff=15
                     )

snr = snr.crop(4 + 4, 4)

# for loop to iterate over masses and find all the maximum SNRs 
# ???

pylab.figure(figsize=[10, 4])
pylab.plot(snr.sample_times, abs(snr))
pylab.ylabel('Signal-to-noise')
pylab.xlabel('Time (s)')
pylab.show()

peak = abs(snr).numpy().argmax()
snrp = snr[peak]
time = snr.sample_times[peak]

print("We found a signal at {}s with SNR {}".format(time, 
                                                    abs(snrp)))

NameError: ignored

2. Whitening of the data

In [11]:
data_whitened = (conditioned.to_frequencyseries() / psd**0.5).to_timeseries()

In [10]:
sum = data_whitened.sum()
mean = sum/len(data_whitened)
print(mean)

-9.754919858927735e-06


So the mean is approximately equal to zero. Let's now plot a histogram of this whitened data to have a look at the distribution of the data (to see if it follows 

In [12]:
import matplotlib.pyplot as plot
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

n_bins = 100
plot = plot.hist(data_whitened, n_bins)
plot

(array([1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 1.10000e+01,
        1.53000e+02, 1.15600e+03, 5.95200e+03, 2.12520e+04,
        5.52040e+04, 9.83650e+04, 1.21451e+05, 1.06381e+05,
        6.47290e+04, 2.72930e+04, 8.07100e+03, 1.68400e+03,
        2.49000e+02, 3.00000e+01, 2.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.0000

We couldn't manage to plot it despite the plot.hist but we've calculated that the mean is almost zero.